<a href="https://colab.research.google.com/github/paramesh25-luv/project1/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from google.colab import files
uploaded = files.upload()


Saving Fake.csv to Fake.csv
Saving True.csv to True.csv


In [11]:
!mkdir -p /content/data/{raw,processed}
!mkdir -p /content/models/{baseline,advanced}
!mkdir -p /content/src

In [12]:
!pip install transformers torch pandas scikit-learn nltk
!python -m nltk.downloader stopwords wordnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [15]:
import shutil
import os

# Assuming the files are uploaded to the default Colab directory
source_path_fake = '/content/Fake.csv'  # Update if your uploaded file has a different name
source_path_true = '/content/True.csv'  # Update if your uploaded file has a different name
destination_path = '/content/data/raw/'

# Move the files
shutil.move(source_path_fake, os.path.join(destination_path, 'Fake.csv'))
shutil.move(source_path_true, os.path.join(destination_path, 'True.csv'))

'/content/data/raw/True.csv'

In [16]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    """
    Clean and preprocess text data by:
    - Removing punctuation
    - Lowercasing
    - Removing generic stopwords (but retaining scientific terms)
    - Lemmatizing words
    """
    if pd.isna(text):
        return ''

    # Remove punctuation and lowercase
    text = re.sub(r'[^\w\s]', '', str(text).lower())
    words = text.split()

    # Retain scientific terms in stopwords
    custom_stopwords = set(stopwords.words('english')) - {'discover', 'species', 'research', 'study'}
    lemmatizer = WordNetLemmatizer()

    # Lemmatize and filter words
    cleaned = [
        lemmatizer.lemmatize(word)
        for word in words
        if word not in custom_stopwords
    ]

    return ' '.join(cleaned)

def main():
    # Load raw datasets
    fake_df = pd.read_csv(os.path.join('data', 'raw', 'Fake.csv'))
    real_df = pd.read_csv(os.path.join('data', 'raw', 'True.csv'))

    # Balance classes by equal sampling
    min_samples = min(len(fake_df), len(real_df))
    fake_df = fake_df.sample(n=min_samples, random_state=42)
    real_df = real_df.sample(n=min_samples, random_state=42)

    # Add labels
    fake_df['label'] = 'FAKE'
    real_df['label'] = 'REAL'

    # Combine and shuffle datasets
    combined_df = pd.concat([fake_df, real_df]).sample(frac=1, random_state=42)

    # Clean text and validate
    combined_df['clean_text'] = combined_df['text'].apply(clean_text)
    combined_df = combined_df[combined_df['clean_text'].str.strip() != '']
    combined_df = combined_df.dropna(subset=['clean_text'])

    # Save processed data
    processed_path = os.path.join('data', 'processed', 'cleaned_news.csv')
    os.makedirs(os.path.dirname(processed_path), exist_ok=True)
    combined_df.to_csv(processed_path, index=False)
    print(f"Balanced dataset saved to {processed_path}")

if __name__ == '__main__':
    main()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Balanced dataset saved to data/processed/cleaned_news.csv


In [17]:
df = pd.read_csv('/content/data/processed/cleaned_news.csv')
df = df.dropna(subset=['clean_text'])
X = df['clean_text']
y = df['label'].map({'FAKE': 0, 'REAL': 1})

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [19]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text (truncate to 128 tokens for memory efficiency)
train_encodings = tokenizer(
    X_train.tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    X_test.tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)

In [22]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Enable GPU acceleration (if available)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = BertForSequenceClassification.from_pretrained('bert-base-uncased').to(device)

# Replace 'evaluation_strategy' with 'eval_steps' for older versions of transformers
training_args = TrainingArguments(
    output_dir='/content/models/advanced/bert_results',
    num_train_epochs=3,
    per_device_train_batch_size=8,  # Reduce if OOM errors occur
    # evaluation_strategy="epoch",
    eval_steps=500, # Evaluate every 500 steps (adjust as needed)
    logging_dir='/content/logs',
    # Disable W&B explicitly
    report_to="none",  # <--- Add this line
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: parameshdurai346 (parameshdurai346-university-college-of-engineering-tindi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


KeyboardInterrupt: 

In [23]:
from transformers import BertForSequenceClassification, BertTokenizer

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
sample_text = "NASA announces discovery of water on Mars."
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, max_length=128)
outputs = model(**inputs)
prediction = torch.argmax(outputs.logits).item()
print("FAKE" if prediction == 0 else "REAL")  # Output: Random guess!

FAKE


# New Section